The colab notebook is accessible at: https://colab.research.google.com/drive/1ejr7tjkDR_5oHMeQ1NrdnJ-EvDJSmCT7?usp=sharing

In [1]:
from sklearn.datasets import load_boston 
ds = load_boston()

# explore the dataset
print(ds.keys())
print(ds.DESCR)

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])
.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX

In [2]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [12]:
X = ds.data
Y = ds.target
m, n = X.shape

# Add a column of 1s to the dataset to account for the bias term
X = np.concatenate([X, np.ones((m, 1))], axis=1)

In [13]:
def solve_weight(X, Y):
    """
      Solve for the weight W using 
      the analytical formula W = (X^T X)^{-1} X^T Y
    """
    Y = Y.reshape(-1, 1)
    w = (np.linalg.inv(X.T @ X) @ X.T) @ Y
    return w


def get_split(X, Y, seed):
    """
      Split the dataset into train and validation (80%/20%) sets
    """
    return train_test_split(X, Y, test_size=0.20, random_state=seed)


def find_residual(y_pred, y_true):
    """
      Computes the residual using a function from sklearn.
        * Note that the mean_squared_error function and the definition 
          of the residual in HW-0 differs by a factor of 2 
    """
    return 0.5 * mean_squared_error(y_true, y_pred)

In [14]:
random_seeds = [0, 1000, 4204]

train_res, val_res = [], []

for seed in random_seeds:
    # Split data and solve for weights
    X_train, X_val, y_train, y_val = get_split(X, Y, seed)
    W = solve_weight(X_train, y_train)
   
    # Make predictions based on learnt weight vector
    y_train_pred = X_train @ W
    y_val_pred = X_val @ W

    # Store residuals
    train_res.append(find_residual(y_train_pred, y_train))
    val_res.append(find_residual(y_val_pred, y_val))

print("Training residuals")
print(train_res)

print("Validation residuals")
print(val_res)

Training residuals
[9.663235101792862, 10.585926141112633, 11.47943598442984]
Validation residuals
[16.724489998843257, 13.041182673583272, 9.462872045433993]


In [15]:
# Some stats
print("Mean and standard deviation of residuals on train data")
print(np.mean(train_res), np.std(train_res))
print("Mean and standard deviation of residuals on validation data")
print(np.mean(val_res), np.std(val_res))

Mean and standard deviation of residuals on train data
10.576199075778446 0.7414928066060814
Mean and standard deviation of residuals on validation data
13.076181572620174 2.9646464114650932
